<a href="https://colab.research.google.com/github/HakureiPOI/Douban_Scraper/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/HakureiPOI/Douban_Scraper.git

Cloning into 'Douban_Scraper'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 12.36 MiB | 13.49 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
!git clone https://github.com/goto456/stopwords.git

Cloning into 'stopwords'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 31 (delta 4), reused 1 (delta 1), pack-reused 22 (from 1)
Receiving objects: 100% (31/31), 22.11 KiB | 5.53 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [4]:
import pandas as pd
import jieba
import nltk
from nltk.corpus import stopwords
import re
import string

In [5]:
nltk.download('punkt')       # 用于英文分词
nltk.download('stopwords')   # 用于获取英文停用词

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
comments = pd.read_excel('/content/Douban_Scraper/comments.xlsx')

In [7]:
comments

,animal,commentator,datetime,rating,content
0,异度侵入 イド：インヴェイデッド ID:INVADED,丘贝贝Kyubei,2020-03-19 21:04:19,还行,写作存在其共性：动画剧本也好，期刊论文与专著也罢，作者一定要有一个认知：不要想着在有限的空间...
1,异度侵入 イド：インヴェイデッド ID:INVADED,晓宇,2020-02-10 21:31:36,力荐,【写在最前边】感谢大家一直以来的点赞、回复、收藏、转发，一路走来大家辛苦了。13话的处理我是...
2,异度侵入 イド：インヴェイデッド ID:INVADED,zx2337,2020-01-27 11:58:34,力荐,把我写在B站的长评也搬过来一份吧看完第5集后，我在评论区发了关于JW真实身份的猜想，讨论了J...
3,异度侵入 イド：インヴェイデッド ID:INVADED,未来事务管理局,2020-04-08 22:17:59,力荐,“一部动画，能够在每集20分钟的螺蛳壳里做出如此多变的领域致敬，可以说是狂热粉丝和匠心制作...
4,异度侵入 イド：インヴェイデッド ID:INVADED,尤莲,2020-03-06 18:33:30,力荐,这两天被强烈安利了《异度侵入ID：INVADED》，男主史称话疗圣手。因而抽空看了。为了让自...
...,...,...,...,...,...
11196,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,岁岁年年花相似,2023-02-04 22:14:02,推荐,闹腾 离谱 欢乐
11197,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,热心市民两块钱,2018-10-07 21:47:56,推荐,这是我的童年没错了 现在看起来海绵宝宝其实很有深度 讽刺了一些社会现象
11198,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,不高興,2021-08-23 22:44:35,推荐,前面有點社會寓言，後面還是蠻好看的，越來越理解到痞老闆很可愛，比基尼海灘永遠不可或缺的反面角...
11199,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,十六太,2022-07-11 19:16:24,推荐,补


In [8]:
def load_stopwords(filepath = '/content/stopwords/hit_stopwords.txt'):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

In [9]:
english_stopwords = set(stopwords.words('english'))

In [10]:
def remove_punctuation(text):
    text = re.sub(r'[{}]+'.format(string.punctuation), '', text)
    text = re.sub(r'[，。！？【】（）《》“”‘’；：、]', '', text)
    return text

In [11]:
def extract_chinese(text):
    chinese_text = re.sub(r'[a-zA-Z0-9]+', '', text)
    return chinese_text

In [12]:
def extract_english(text):
    english_text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return english_text

In [13]:
def process_text(text, chinese_stopwords, english_stopwords):
    text = remove_punctuation(text)

    chinese_text = extract_chinese(text)
    english_text = extract_english(text)

    chinese_words = jieba.cut(chinese_text)
    english_words = nltk.word_tokenize(english_text)

    all_words = list(chinese_words) + english_words

    filtered_words = [word for word in all_words
                      if word not in chinese_stopwords and word.lower() not in english_stopwords]

    return filtered_words

In [20]:
def process_row(row, chinese_stopwords, english_stopwords):
    text = str(row['content'])
    filtered_words = process_text(text, chinese_stopwords, english_stopwords)

    if len(filtered_words) == 0:
        return

    with open('filtered_words.txt', 'a', encoding='utf-8') as f:
        f.write(' '.join(filtered_words) + '\n')
        f.close()

In [21]:
chinese_stopwords = load_stopwords()

In [22]:
for index, row in comments.iterrows():
    process_row(row, chinese_stopwords, english_stopwords)

In [23]:
!cp /content/filtered_words.txt /content/drive/MyDrive/HITSZ/大数据/filtered_words.txt